# 回顾

In [3]:
deployment="gpt4" #在azure中的deployment name
model="gpt-4"

In [12]:
def translate(text):
    messages = []
    messages.append( {"role": "system", 
                      "content": "You are a translator. Please, translate the user's request to English."})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        engine=deployment, 
        model=model,
        messages=messages,
        temperature=0.5,
        max_tokens = 100
    )
    return response["choices"][0]["message"]["content"]

In [13]:
import openai
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")

print(encoding)

chinese = """在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。"""
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))

print(f"chinese:{chinese} ; {num_of_tokens_in_chinese} tokens\n")
print(f"english:{english} ; {num_of_tokens_in_english} tokens\n")

<Encoding 'cl100k_base'>
chinese:在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。 ; 35 tokens

english:Before the future arrives, there always needs to be someone to create it, and that person should be us. ; 22 tokens



# 常见应用场景

## 意图识别

In [15]:
import openai
response = openai.ChatCompletion.create(
    engine=deployment, # 如果直接访问OpenAI GPT服务的同学，这里不要使用engine这个参数，要使用model，如： model=“gpt-4”
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input 
         """},
        #{"role": "user", "content": "订明天早5点北京到上海的飞机"}
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

设置提醒


## 生成SQL

In [17]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can anwser the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] 
                  """

prompt = system_prompt

response = openai.ChatCompletion.create(
    engine=deployment, # # 如果直接访问OpenAI GPT服务的同学，这里不要使用engine这个参数，要使用model，如： model=“gpt-4”
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"},
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

首先，我们需要找到英语课程的id，然后在学生表中找到所有参加这门课程的学生的成绩，然后计算平均值。以下是SQL查询：

```sql
SELECT AVG(score) 
FROM students 
WHERE course_id = (
    SELECT id 
    FROM courses 
    WHERE name = '英语'
)
```

这个查询首先在courses表中找到名为'英语'的课程的id，然后在students表中找到所有参加这门课程的学生的成绩，然后计算这些成绩的平均值。


## 生成代码友好的提示词

### 规范输出的格式

In [18]:
import openai
response = openai.ChatCompletion.create(
    engine=deployment, # engine = "deployment_name".
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input and format output as JSON string. 
        The output JSON string includes: "intention", "paramters" """},
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

{"intention": "set_reminder", "parameters": {"time": "tomorrow 8am", "event": "meeting"}}


In [19]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

prompt = system_prompt

response = openai.ChatCompletion.create(
    engine=deployment,
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        #{"role": "user", "content": "列出英语成绩大于80分的学生"},
        {"role": "user", "content": "列出英语课程成绩大于80分的学生, 返回结果只包括SQL"},
        #{"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

SELECT students.name 
FROM students 
JOIN courses ON students.course_id = courses.id 
WHERE courses.name = '英语' AND students.score > 80;


### 文本规范异常输出的格式 

In [22]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

system_prompt_with_negative =  """  
You are a software engineer, you can write a SQL string as the anwser according to the user request.
Also, when you cannot create the SQL query for the user's request based on the given tables, please, only return "invalid request"
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

#prompt = system_prompt
prompt = system_prompt_with_negative

response = openai.ChatCompletion.create(
    engine=deployment,
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

-1
